In [1]:
%pip install python-dotenv jupysql duckdb duckdb-engine sqlalchemy openai twitter-api-client --quiet

Note: you may need to restart the kernel to use updated packages.


# Elementus API Usage Example

In [1]:
from elementus import ElementusClient, ElementusAPIError
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
from helper import *
import os

# Load the environment variables
load_env()

# Retrieve Google BigQuery connection string from the environment
GBQ_URI = get_gbq_uri()
engine = create_engine(GBQ_URI)

# Retrieve the API key from the environment
ELEMENTUS_API_KEY = get_elementus_api_key()

/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### Retrieve last Bitcoin block transactions over 1 BTC

In [2]:
%load_ext sql
%sql engine

In [3]:
%%sql result <<
WITH max_block AS (
  SELECT MAX(number) as max_number 
  FROM `bigquery-public-data.crypto_bitcoin.blocks`
),
inputs AS (
  SELECT DISTINCT
    transaction_hash AS tx_hash,
    addresses
  FROM `bigquery-public-data.crypto_bitcoin.inputs`, max_block
  WHERE block_number = max_number
),
outputs AS (
  SELECT 
    transaction_hash AS tx_hash,
    addresses[OFFSET(0)] AS address,
    value / 100000000 AS value
  FROM `bigquery-public-data.crypto_bitcoin.outputs`, max_block
  WHERE block_number = max_number
)
SELECT 
    o.tx_hash, 
    o.address AS output_address, 
    o.value,
    ARRAY_AGG(DISTINCT input_address) AS input_addresses
FROM outputs o
JOIN inputs i ON o.tx_hash = i.tx_hash
CROSS JOIN i.addresses AS input_address
WHERE o.address NOT IN UNNEST(i.addresses) AND 
o.value >= 1
GROUP BY o.tx_hash, o.address, o.value
ORDER BY value DESC

Running query in 'bigquery://elementus-prod-242016'

199 rows affected.

In [4]:
df = result.DataFrame()
df

,tx_hash,output_address,value,input_addresses
0,9b37bbe40cd79cf216649e3034d2365b7ff8a26e2ea041...,bc1qk7zrv26d3q8f8yfgalu28cdn78uvfghtuttfjj,848.00611855,[1GYsWMeLC1LyTS7nUPigmXA8jWo9nyMkXh]
1,1a45bf848659cee23db925d834196e0652bbd4f2f254e4...,36TFyGRTWuqbropfXFETFZKH8DizCchoNo,633.57899693,[3HBeuYgjyXRvPpicCm9RUcaeHVqzyY2RQY]
2,7f1b2ed56a375aa69876564a91d14cd25f834bb2dfcc06...,3CaKdd1uoNEt543qh959nahk1sPFjVUrge,623.57898868,[36TFyGRTWuqbropfXFETFZKH8DizCchoNo]
3,0d33b269a6fcd0b22d2971a635c2bff95736ca33c9578a...,bc1qf4q4ws0chmhte2ydl6cfgkpaq25sgu80et5278,579.998988,"[bc1qkz0uf4jpupzl7tzfsn00rpygchmyeu6k5c3m3c, b..."
4,b4106907a3b84916f71028060b5adb4fd8575eba7e75d3...,12Apv6b8qSZh4Nwr7H8cm8NZMxGsCCWfv6,531.69797719,[1A5jkWVY4Rszt1Htg29qH8RWupS8GxEUJw]
...,...,...,...,...
194,1db1735de2395199fb3a30aaac816b79397cd23f9f6d9f...,3PS8VjDCckxsLJD9dGvzxkiwiJVCGTkkzp,1,[bc1qq79qrlcsjmrhquyfyv672nhanmwlcvmu6fqn4c]
195,ef8421314260eee35223ebb59f7aab68b7c84e07723fc7...,39wcynLpZNGDkS99VrgAne8heq3SnneFzr,1,[bc1qpqf4py9heypkrpsu5c2qmn052238v7wjvr5856]
196,1b0409c034c356c1558a49bd8123a9061029e03ae598ec...,34cE33tjpXVEvVk89PsG3GHVxD9HJS5acJ,1,[bc1qt7803rml0gj8nxc4nnxd5tpsgan08pyga05uqu]
197,c1ca4648fdd1bc59cf325ff11d006603dc2e0182dce65b...,3PS8VjDCckxsLJD9dGvzxkiwiJVCGTkkzp,1,[bc1qq79qrlcsjmrhquyfyv672nhanmwlcvmu6fqn4c]


### Use Elementus API to retrieve entity attributions for input and output addresses

In [5]:
client = ElementusClient(api_key=ELEMENTUS_API_KEY)

# Check API health
is_healthy = await client.check_health()

output_addresses = df['output_address'].unique().tolist()
input_addresses = list(set(addr for sublist in df['input_addresses'] for addr in sublist))

try:
    response = await client.get_address_attributions(output_addresses)
except ElementusAPIError as e:
    print(f"API error: {e}")

# Create a dictionary from the response data
attribution_dict = {address: attr.entity for address, attr in response.data.items()}

try:
    response = await client.get_address_attributions(input_addresses)
except ElementusAPIError as e:
    print(f"API error: {e}")

attribution_dict.update({address: attr.entity for address, attr in response.data.items()})

len(attribution_dict)

234

### Add entity attributions to transaction data

In [6]:
df['output_entity'] = df['output_address'].map(attribution_dict)
df['input_entity'] = [
    next((attribution_dict[key] for key in addresses if key in attribution_dict), None)
    for addresses in df['input_addresses']
]

df.dropna(subset=['input_entity', 'output_entity'], how='all', inplace=True)
df = df[df['input_entity'] != df['output_entity']]
# Ensure 'input_entity' is a string type before using str.startswith
df = df[~df['input_entity'].astype(str).str.startswith('fastmoney_') & ~df['output_entity'].astype(str).str.startswith('fastmoney_')]

df

,tx_hash,output_address,value,input_addresses,output_entity,input_entity
0,9b37bbe40cd79cf216649e3034d2365b7ff8a26e2ea041...,bc1qk7zrv26d3q8f8yfgalu28cdn78uvfghtuttfjj,848.00611855,[1GYsWMeLC1LyTS7nUPigmXA8jWo9nyMkXh],NaN,shakepay.com
6,b7f729066163bbdbf2e3b0c780425e9f63453302de39b9...,1AcwwfWLCUnfxSbUh3VEqULmZCFVBCwaAC,400.57019312,"[3MqUP6G1daVS5YTD8fz3QgwjZortWwxXFd, 3DXFaC8KS...",NaN,coinbase_prime
7,544d41b9b989098119d99e42e4b2a1df17a4b0659a11a5...,bc1qd0yw8wsqczelsvxua6wm9n9jn9c457na95nr93nyqk...,112.34179247,"[3DYwNSCLATfc7TYMg2t4nqtntPtEDs8hds, 32retgqZV...",NaN,kraken.com
9,a8a4f2dcf34698f2de858d600e596f037dbc1ca16aa8f6...,1Ghn3eH6BkLdFFTveC91YFXMVERQzy2pQY,91.6,[bc1qm34lsc65zpw79lxes69zkqmk6ee3ewf0j77s3h],wintermute.com,binance.com
12,2c5ae3c6aa0b5b9ed74e9e65de16a0c62565ff19797b2a...,bc1qj64fwdeaej670gj0vag9d99t2rlu7rd4ssjwj77jwu...,81.05598382,"[39YGmrxhAQrSfFobE368vXWBkxHkxAEdyC, 3Gcuo1kdj...",NaN,kraken.com
14,096133bf5630cafee7175c9d917a4f6eebc82e72197bfd...,bc1qks3ykhe02q5wugsg4dlqyk5mm7tr7hv7lj6v67265u...,79.26035177,[37PtiwXH1Xj2Fqaq8zNL6evGEXHspzwojM],NaN,kraken.com
17,3dd8eb86f02546a7c2e11bdb3b6aad03e8599e3241af3b...,bc1qk8lw890red4mrtvd065nzwh7dk7ru968rds2re,72.95858255,"[bc1qhk6dk3mng6kc2fwcqpv27xaxy3ygftagt0v5gj, 1...",falconx.io,wintermute.com
24,07c9a491ca7f9cef9e90f9206423f892255ca60c7f789e...,bc1qmxcagqze2n4hr5rwflyfu35q90y22raxdgcp4p,54.3866811,[bc1q8s3h3vw5xufdas890q29lpuca56r0ezqar0mvs],robinhood.com,cumberland.io
29,ad550d018d20f5655ddab6e7fd8496ee20fdbd5850117b...,345j2JWoSaDTwDAB14quFHjovgmkY8mFRh,47.7,[bc1qddvmdc9t8tad2eqd7vfwv2mexjzs49f42wvg27],bitfinex.com,wintermute.com
30,f8a435bc47bfdedbc437a022bffa880ccec6a5c4eb7a41...,1KbDEg1tDz2ErYgaDbaDhhawnLrSQFaFx5,46.5,[1Kr6QSydW9bFQG1mXiPNNu6WpJGmUa9i1g],binance.com,bitfinex.com


In [7]:
# Convert DataFrame to JSON string
json_string = df[['tx_hash', 'input_entity', 'output_entity', 'value']].to_json(orient='records')

json_string

'[{"tx_hash":"9b37bbe40cd79cf216649e3034d2365b7ff8a26e2ea0418875cf159a68b72fb9","input_entity":"shakepay.com","output_entity":null,"value":848.00611855},{"tx_hash":"b7f729066163bbdbf2e3b0c780425e9f63453302de39b9db1556926f0eee8b2e","input_entity":"coinbase_prime","output_entity":null,"value":400.57019312},{"tx_hash":"544d41b9b989098119d99e42e4b2a1df17a4b0659a11a544c87b38cea86a9a7e","input_entity":"kraken.com","output_entity":null,"value":112.34179247},{"tx_hash":"a8a4f2dcf34698f2de858d600e596f037dbc1ca16aa8f657c26e4f4e227109a8","input_entity":"binance.com","output_entity":"wintermute.com","value":91.6},{"tx_hash":"2c5ae3c6aa0b5b9ed74e9e65de16a0c62565ff19797b2a25e41652daf7f72973","input_entity":"kraken.com","output_entity":null,"value":81.05598382},{"tx_hash":"096133bf5630cafee7175c9d917a4f6eebc82e72197bfdf393b5cb4e3d8f44d5","input_entity":"kraken.com","output_entity":null,"value":79.26035177},{"tx_hash":"3dd8eb86f02546a7c2e11bdb3b6aad03e8599e3241af3bff2b358cd2d21a02a9","input_entity":"w

### Create X Post using X AI to generate on-chain insights from transaction data

In [8]:
from openai import OpenAI
from IPython.display import display, Markdown

OPENAI_API_KEY = get_openai_api_key()

client = OpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.x.ai/v1",
    #base_url="https://api.openai.com/v1",
)

completion = client.chat.completions.create(
    model="grok-2-latest",
    #model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": "You are an expert in Bitcoin blockchain analytics and market structure"
        },
        {
            "role": "user",
            "content": "Given a list of Bitcoin transactions attributed to entities and news currently trending on Crypto Twitter create an X post explaining the most inportant on-chain insights, which may affect BTC/USD price. Provide only the text ready to be sent via X API without any additional comments. Limit the text to 160 characters. " + json_string
        },
    ],
    temperature=0.8,
)

display(Markdown(f"## X Post\n{completion.choices[0].message.content}"))


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x17b285af0>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x17b8a7650>, 97431.167218583)])']
connector: <aiohttp.connector.TCPConnector object at 0x17b2859d0>


## X Post
Large outflows from exchanges like Coinbase and Kraken, with significant BTC moving to Wintermute and FalconX. This might signal institutional accumulation, potentially bullish for BTC/USD. #Bitcoin #OnChainAnalysis

### Tweet the generated post

In [9]:
from twitter.account import Account

account = Account(cookies={"ct0": "", "auth_token": ""})
account.tweet(text=completion.choices[0].message.content)

/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.12/site-packages/twitter/account.py:778: SyntaxWarning: invalid escape sequence '\d'
  user_id = int(re.findall('"u=(\d+)"', self.session.cookies.get('twid'))[0])
/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.12/site-packages/twitter/account.py:793: SyntaxWarning: invalid escape sequence '\d'
  user_id = int(re.findall('"u=(\d+)"', self.session.cookies.get('twid'))[0])
/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.12/site-packages/twitter/account.py:832: SyntaxWarning: invalid escape sequence '\d'
  return int(re.findall('"u=(\d+)"', self.session.cookies.get('twid'))[0])


Exception: Session not authenticated. Please use an authenticated session or remove the `session` argument and try again.